In [ ]:
from dotenv import load_dotenv
import locale
import numpy as np
import os
import pandas as pd
import sqlalchemy as sa

load_dotenv()

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

engine: sa.Engine = sa.create_engine(os.getenv("URL_MYSQL"))

In [ ]:
new: list[dict[str: int | str | bool]] = [
    # {"concurso": 2747, "data": "2024-07-10", "bolas": "14 17 24 28 36 45", "ganhou": False},
]

rows_inserted: int = pd.DataFrame(new).to_sql(name="megasena", con=engine, if_exists="append", index=False)
print(f"Foram {rows_inserted} jogo(s) inserido(s) com sucesso!")

In [ ]:
df_last: pd.DataFrame = pd.read_sql(sql=sa.text("SELECT * FROM megasena"), con=engine)

df_last["concurso"] = df_last["concurso"].astype(str).str.zfill(4)
df_last["data"] = pd.to_datetime(df_last["data"]).dt.strftime("%x (%a)")
df_last["ganhou"] = np.where(df_last["ganhou"], "Alguém ganhou", "Ninguém ganhou")
df_last.columns = ["Concurso", "Data", "Bolas", "Ganhou?"]

df_last.tail(15)

In [ ]:
minhas_apostas: tuple = (
    "05 15 26 27 46 53",  # aposta n.° 1
    "03 12 19 20 45 47",  # aposta n.° 2
    "01 10 17 41 42 56",  # aposta n.° 3
    "02 10 13 27 53 55",  # aposta n.° 4
    "06 07 08 11 43 56",  # aposta n.° 5
    "08 10 14 25 33 34",  # aposta n.° 6
    "05 11 16 40 43 57",  # aposta n.° 7
    "04 05 08 13 17 38",  # aposta n.° 8
    "13 24 32 49 51 60",  # aposta n.° 9
    "11 16 19 43 58 60",  # aposta n.° 10
    "03 05 10 20 35 46",  # aposta n.° 11
    "02 09 10 19 31 57",  # aposta n.° 12
    "04 18 20 21 39 57",  # aposta n.° 13
    "02 11 22 36 49 60",  # aposta n.° 14
    "02 21 39 48 52 57",  # aposta n.° 15
    "14 41 45 50 54 59",  # aposta n.° 16
    "13 20 22 25 28 39",  # aposta n.° 17
    "01 16 21 34 49 54",  # aposta n.° 18
)

In [ ]:
pd.read_sql(sql=sa.text(f"SELECT * FROM megasena WHERE bolas IN {minhas_apostas}"), con=engine)

In [ ]:
mega: dict[str: list] = {"Concurso": [], "Data": [], "Bolas": [], "Aposta n.°": [], "Acertos": []}

for row in pd.read_sql(sql=sa.text("SELECT * FROM megasena"), con=engine).itertuples(index=False, name=None):
    for aposta in minhas_apostas:
        bolas: list[str] = aposta.split()
        match: list[str] = []
        for x in range(6):
            if bolas[x] in row[2]:
                match.append(bolas[x])
        if len(match) >= 4:
            mega["Concurso"].append(str(row[0]).zfill(4))
            mega["Data"].append(pd.to_datetime(row[1]).strftime("%x (%a)"))
            mega["Bolas"].append(" ".join(match))
            mega["Aposta n.°"].append(minhas_apostas.index(aposta) + 1)
            mega["Acertos"].append(len(match))

pd.DataFrame(mega)

In [ ]:
sua_aposta: str = "12 14 31 45 50 60"

mega: dict[str: list] = {"Concurso": [], "Data": [], "Bolas": [], "Acertos": []}

for row in pd.read_sql(sql=sa.text("SELECT * FROM megasena"), con=engine).itertuples(index=False, name=None):
    match: list[str] = []
    for aposta in sua_aposta.split():
        if aposta in row[2]:
            match.append(aposta)
    if len(match) >= 4:
        mega["Concurso"].append(str(row[0]).zfill(4))
        mega["Data"].append(pd.to_datetime(row[1]).strftime("%x (%a)"))
        mega["Bolas"].append(row[2])
        mega["Acertos"].append(len(match))

pd.DataFrame(mega)

In [ ]:
stmt:str = "SELECT * FROM megasena WHERE data IN (SELECT MAX(data) FROM megasena GROUP BY YEAR(data) HAVING YEAR(data) <> YEAR(CURRENT_DATE))"

df_mega_da_virada: pd.DataFrame = pd.read_sql(sql=sa.text(stmt), con=engine)
df_mega_da_virada["concurso"] = df_mega_da_virada["concurso"].astype(str).str.zfill(4)
df_mega_da_virada["data"] = pd.to_datetime(df_mega_da_virada["data"]).dt.strftime("%x (%a)")
df_mega_da_virada["ganhou"] = np.where(df_mega_da_virada["ganhou"], "Alguém ganhou", "Ninguém ganhou")
df_mega_da_virada.columns = ["Concurso", "Data", "Bolas", "Ganhou?"]
df_mega_da_virada

In [ ]:
apostas: dict[int: int] = {}

for x in minhas_apostas:
    y: list[str] = x.split()
    for z in range(len(y)):
        try:
            apostas[int(y[z])] += 1
        except KeyError:
            apostas[int(y[z])] = 1

apostas_ordenadas: dict[int: int] = {k: v for k, v in sorted(apostas.items(), key=lambda r: r[0])}

acertos: pd.DataFrame = pd.DataFrame(data=apostas_ordenadas, index=["acertos"], columns=apostas_ordenadas.keys())
acertos.index.name = "bolas"
acertos